Wave 0: Slope based tests

In [1]:
# Imports
import numpy as np
from river import drift
import statistics
import matplotlib.pyplot as plt

In [2]:
# Parameters
mu = [0.01, 0.05, 0.1, 0.5, 1]#0.1
n = 1000
k = 50
slope = [0.001, 0.002, 0.004, 0.01, 1] 
#delta = [0.0005, 0.001, 0.002, 0.005, 0.01]
o = 0.05
size = 10

In [3]:
# Generate data for 3 distributions
print("Generating Streams...")
random_state = np.random.RandomState(seed=42)

streams = {}

for s in slope:

    streams[s] = {}

    for u in mu:

        streams[s][u] = []

        for _i in range(size):

            stream = np.array([])
            width = int(1 / s)
            for i in range(k):
                up = [stream]

                for i in range(width):
                    up.append(random_state.normal(0.2+0.1*(i/width), u, 1))

                up.append(random_state.normal(0.3, u, n - width))

                for i in reversed(range(width)):
                    up.append(random_state.normal(0.2+0.1*(i/width), u, 1))

                up.append(random_state.normal(0.2, u, n - width))

                stream = np.concatenate(up)

            streams[s][u].append(stream)
        
        #plt.plot(stream)
        #plt.show()
print("Stream Generated")

Generating Streams...
Stream Generated


In [4]:
# Functions

def average(data):
    data = [i for i in data if i!=0]
    if len(data) > 0:
        mean = round(statistics.mean(data), 1)
        sd = 0
        if len(data) > 1:
            sd = round(statistics.stdev(data), 1)
        out = f"{mean} ± {sd}"
        return out
    else:
        return "Null"

In [14]:
def test_it(s):
    delta_result = []
    delta_result2 = []

    detection_rate = []
    detection_rate2 = []

    print(f"Slope = {s}")

    _streams = streams[s]

    for u in mu:

        _stream = _streams[u]

        print(f"Mu = {u}")

        # Adwin
        drifts = [0] * k * 2 * size
        drift_counts = []

        for s, stream in enumerate(_stream):
            drift_detector = drift.ADWIN(delta=o)
            drift_count = 0

            for i, val in enumerate(stream):
                if i % n == 0 :
                    print(f"\rProgress: {i}", end = '\r')
                drift_detector.update(val)   # Data is processed one sample at a time

                if drift_detector.drift_detected:
                    # The drift detector indicates after each sample if there is a drift in the data
                    #print(f'Change detected at index {i} by ADWIN')
                    #drifts.append(i)
                    drift_count += 1
                    current = i // n
                    if drifts[current + (s * k * 2)] == 0:
                        drifts[current + (s * k * 2)] = i % n
            drift_counts.append(drift_count)

        print("ADWIN Complete     ")

        print("Results:")
        print(f"ADWIN Drifts: {average(drift_counts)}")
        print(f"Average Detection Time {average(drifts)}")

        delta_result.append(drifts)
        detection_rate.append(drift_counts)

        # Atwin
        test_drift_counts = []
        test_drifts = [0] * k * 2 * size

        for s, stream in enumerate(_stream):
            test_drift_detector = drift.atwin.ATWIN(delta=o, temp=1/100)
            test_drift_count = 0

            for i, val in enumerate(stream):
                if i % n == 0 :
                    print(f"\rProgress: {i}", end = '\r')
                test_drift_detector.update(val)

                if test_drift_detector.drift_detected:
                    test_drift_count += 1
                    current = i // n
                    if test_drifts[current + (s * k * 2)] == 0:
                        test_drifts[current + (s * k * 2)] = i % n
            test_drift_counts.append(test_drift_count)

        print("ATWIN Complete     ")

        print("Results:")
        print(f"ATWIN Drifts: {average(test_drift_counts)}")
        print(f"Average Detection Time {average(test_drifts)}")

        delta_result2.append(test_drifts)
        detection_rate2.append(test_drift_counts)

        """
        plt.plot(stream)
        for _d in _drifts:
            plt.axvline(_d, color='red')
        for _d in _test_drifts:
            plt.axvline(_d, color='blue')
        plt.show()
        """
        
    return delta_result, delta_result2, detection_rate, detection_rate2

In [15]:
slope_result = []
slope_result2 = []
d_rate = []
d_rate2 = []

"""
for s in slope:
    a, b = test_it(s)
    slope_result.append(a)
    slope_result2.append(b)
"""

'\nfor s in slope:\n    a, b = test_it(s)\n    slope_result.append(a)\n    slope_result2.append(b)\n'

In [16]:
s = slope[0]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.001
0.001 0.01
Mu = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 701.5 ± 15.6
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 698.7 ± 15.7
9 0.05
Mu = 0.05
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 747.3 ± 34.5
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 744.5 ± 34.2
9 0.1
Mu = 0.1
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 829.1 ± 81.3
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 822.2 ± 80.3
9 0.5
Mu = 0.5
ADWIN Complete     
Results:
ADWIN Drifts: 6 ± 3.7
Average Detection Time 517.3 ± 230.3
ATWIN Complete     
Results:
ATWIN Drifts: 22.7 ± 11.0
Average Detection Time 459.4 ± 286.3
9 1
Mu = 1
ADWIN Complete     
Results:
ADWIN Drifts: 1.4 ± 0.5
Average Detection Time 425.3 ± 253.7
ATWIN Complete     
Results:
ATWIN Drifts: 173.9 ± 73.6
Average Detection Time 311.1 ± 252.7


In [17]:
s = slope[1]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.002
0.002 0.01
Mu = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 352.3 ± 20.2
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 350.3 ± 20.2
9 0.05
Mu = 0.05
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 389.3 ± 27.1
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 386.7 ± 26.3
9 0.1
Mu = 0.1
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 459.0 ± 55.8
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 453.6 ± 54.5
9 0.5
Mu = 0.5
ADWIN Complete     
Results:
ADWIN Drifts: 24.7 ± 4.6
Average Detection Time 407.2 ± 282.1
ATWIN Complete     
Results:
ATWIN Drifts: 53.2 ± 17.3
Average Detection Time 456.9 ± 291.4
9 1
Mu = 1
ADWIN Complete     
Results:
ADWIN Drifts: 3.4 ± 3.7
Average Detection Time 416.8 ± 251.5
ATWIN Complete     
Results:
ATWIN Drifts: 190.5 ± 63.4
Average Detection Time 321.4 ± 260.5


In [18]:
s = slope[2]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.004
0.004 0.01
Mu = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 220.7 ± 25.0
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 218.1 ± 25.3
9 0.05
Mu = 0.05
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 256.2 ± 28.0
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 252.1 ± 27.5
9 0.1
Mu = 0.1
ADWIN Complete     
Results:
ADWIN Drifts: 99.8 ± 0.4
Average Detection Time 309.3 ± 46.0
ATWIN Complete     
Results:
ATWIN Drifts: 99.8 ± 0.4
Average Detection Time 302.7 ± 42.9
9 0.5
Mu = 0.5
ADWIN Complete     
Results:
ADWIN Drifts: 31.7 ± 5.1
Average Detection Time 482.6 ± 320.4
ATWIN Complete     
Results:
ATWIN Drifts: 77.6 ± 11.0
Average Detection Time 478.3 ± 274.2
9 1
Mu = 1
ADWIN Complete     
Results:
ADWIN Drifts: 3.7 ± 2.3
Average Detection Time 418.2 ± 271.4
ATWIN Complete     
Results:
ATWIN Drifts: 214 ± 34.6
Average Detection Time 301.4 ± 260.5


In [19]:
s = slope[3]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.01
0.01 0.01
Mu = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 99.1 ± 0.3
Average Detection Time 144.9 ± 25.8
ATWIN Complete     
Results:
ATWIN Drifts: 99.1 ± 0.3
Average Detection Time 141.0 ± 25.9
9 0.05
Mu = 0.05
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 169.5 ± 14.6
ATWIN Complete     
Results:
ATWIN Drifts: 99 ± 0.0
Average Detection Time 165.2 ± 14.8
9 0.1
Mu = 0.1
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 212.9 ± 30.8
ATWIN Complete     
Results:
ATWIN Drifts: 99 ± 0.0
Average Detection Time 205.6 ± 30.0
9 0.5
Mu = 0.5
ADWIN Complete     
Results:
ADWIN Drifts: 34.2 ± 5.6
Average Detection Time 530.7 ± 320.3
ATWIN Complete     
Results:
ATWIN Drifts: 77 ± 6.4
Average Detection Time 461.9 ± 269.8
9 1
Mu = 1
ADWIN Complete     
Results:
ADWIN Drifts: 3.4 ± 3.4
Average Detection Time 487.8 ± 300.8
ATWIN Complete     
Results:
ATWIN Drifts: 219.2 ± 26.2
Average Detection Time 316.6 ± 258.2


In [20]:
s = slope[4]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 1
1 0.01
Mu = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 90.4 ± 9.6
ATWIN Complete     
Results:
ATWIN Drifts: 99 ± 0.0
Average Detection Time 87.5 ± 9.8
9 0.05
Mu = 0.05
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 116.5 ± 16.1
ATWIN Complete     
Results:
ATWIN Drifts: 99 ± 0.0
Average Detection Time 110.7 ± 16.5
9 0.1
Mu = 0.1
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 156.6 ± 27.6
ATWIN Complete     
Results:
ATWIN Drifts: 99 ± 0.0
Average Detection Time 149.4 ± 26.8
9 0.5
Mu = 0.5
ADWIN Complete     
Results:
ADWIN Drifts: 34.1 ± 6.7
Average Detection Time 564.5 ± 311.0
ATWIN Complete     
Results:
ATWIN Drifts: 81.7 ± 10.6
Average Detection Time 429.1 ± 271.7
9 1
Mu = 1
ADWIN Complete     
Results:
ADWIN Drifts: 2.6 ± 1.5
Average Detection Time 419.9 ± 267.5
ATWIN Complete     
Results:
ATWIN Drifts: 233 ± 21.2
Average Detection Time 297.2 ± 244.4


In [21]:
for s, d , d2 in zip(slope, slope_result, slope_result2):
    print("\multirow{2}{2em}{" + str(s) + "}", end="")
    print(" & ADWIN & " + " & ".join([average(t) for t in d]) + " \\\\")
    print(" & ATWIN & " + " & ".join([average(t) for t in d2]) + " \\\\")
    print("\hline")

\multirow{2}{2em}{0.001} & ADWIN & 701.5 ± 15.6 & 747.3 ± 34.5 & 829.1 ± 81.3 & 517.3 ± 230.3 & 425.3 ± 253.7 \\
 & ATWIN & 698.7 ± 15.7 & 744.5 ± 34.2 & 822.2 ± 80.3 & 459.4 ± 286.3 & 311.1 ± 252.7 \\
\hline
\multirow{2}{2em}{0.002} & ADWIN & 352.3 ± 20.2 & 389.3 ± 27.1 & 459.0 ± 55.8 & 407.2 ± 282.1 & 416.8 ± 251.5 \\
 & ATWIN & 350.3 ± 20.2 & 386.7 ± 26.3 & 453.6 ± 54.5 & 456.9 ± 291.4 & 321.4 ± 260.5 \\
\hline
\multirow{2}{2em}{0.004} & ADWIN & 220.7 ± 25.0 & 256.2 ± 28.0 & 309.3 ± 46.0 & 482.6 ± 320.4 & 418.2 ± 271.4 \\
 & ATWIN & 218.1 ± 25.3 & 252.1 ± 27.5 & 302.7 ± 42.9 & 478.3 ± 274.2 & 301.4 ± 260.5 \\
\hline
\multirow{2}{2em}{0.01} & ADWIN & 144.9 ± 25.8 & 169.5 ± 14.6 & 212.9 ± 30.8 & 530.7 ± 320.3 & 487.8 ± 300.8 \\
 & ATWIN & 141.0 ± 25.9 & 165.2 ± 14.8 & 205.6 ± 30.0 & 461.9 ± 269.8 & 316.6 ± 258.2 \\
\hline
\multirow{2}{2em}{1} & ADWIN & 90.4 ± 9.6 & 116.5 ± 16.1 & 156.6 ± 27.6 & 564.5 ± 311.0 & 419.9 ± 267.5 \\
 & ATWIN & 87.5 ± 9.8 & 110.7 ± 16.5 & 149.4 ± 26.8 & 429.

In [22]:
for s, r, r2 in zip(slope, d_rate, d_rate2):
    print("\multirow{2}{2em}{" + str(s) + "}", end="")

    temp = [str(average([round(100 * (t / (2 * k)), 0) for t in _r])) for _r in r]
    print(" & ADWIN & " + " & ".join(temp) + " \\\\")

    temp2 = [str(average([round(100 * (t / (2 * k)), 0) for t in _r])) for _r in r2]
    print(" & ATWIN & " + " & ".join(temp2) + " \\\\")
    print("\hline")

\multirow{2}{2em}{0.001} & ADWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 & 6.0 ± 3.7 & 1.4 ± 0.5 \\
 & ATWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 & 22.7 ± 11.0 & 173.9 ± 73.6 \\
\hline
\multirow{2}{2em}{0.002} & ADWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 & 24.7 ± 4.6 & 3.4 ± 3.7 \\
 & ATWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 & 53.2 ± 17.3 & 190.5 ± 63.4 \\
\hline
\multirow{2}{2em}{0.004} & ADWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 99.8 ± 0.4 & 31.7 ± 5.1 & 3.7 ± 2.3 \\
 & ATWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 99.8 ± 0.4 & 77.6 ± 11.0 & 214.0 ± 34.6 \\
\hline
\multirow{2}{2em}{0.01} & ADWIN & 99.1 ± 0.3 & 99.0 ± 0.0 & 99.0 ± 0.0 & 34.2 ± 5.6 & 3.4 ± 3.4 \\
 & ATWIN & 99.1 ± 0.3 & 99.0 ± 0.0 & 99.0 ± 0.0 & 77.0 ± 6.4 & 219.2 ± 26.2 \\
\hline
\multirow{2}{2em}{1} & ADWIN & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 34.1 ± 6.7 & 2.6 ± 1.5 \\
 & ATWIN & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 81.7 ± 10.6 & 233.0 ± 21.2 \\
\hline
